# RNN Prediction Model for crimes in the México City

## Setting up

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 500)


#TensorFlow
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.callbacks import EarlyStopping


### URL to get data set

In [22]:
url = "https://storage.googleapis.com/safety-map-model/preprocessed_data.csv"
df = pd.read_csv(url)
df.drop(columns = "Neighborhood_ID", inplace = True)
df

,Neighborhood,year_month,burglary,danger of well-being,domestic violence,fraud,homicide,property damage,robbery with violence,robbery without violence,sexual crime,threats
0,ALVARO OBREGON || 16 DE SEPTIEMBRE,2019-01-01,0,0,0,0,0,0,0,0,0,0
1,ALVARO OBREGON || 16 DE SEPTIEMBRE,2019-02-01,0,0,0,0,0,0,0,0,0,0
2,ALVARO OBREGON || 16 DE SEPTIEMBRE,2019-03-01,0,0,0,0,0,0,0,0,0,0
3,ALVARO OBREGON || 16 DE SEPTIEMBRE,2019-04-01,0,0,0,0,0,0,0,0,0,0
4,ALVARO OBREGON || 16 DE SEPTIEMBRE,2019-05-01,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
104086,XOCHIMILCO || XICALHUACAN,2022-11-01,1,0,0,0,0,0,0,0,0,1
104087,XOCHIMILCO || XICALHUACAN,2022-12-01,0,0,0,0,0,0,0,1,0,1
104088,XOCHIMILCO || XICALHUACAN,2023-01-01,0,1,1,0,0,1,0,0,0,1
104089,XOCHIMILCO || XICALHUACAN,2023-02-01,0,0,0,0,0,0,0,0,0,0


In [37]:
df.columns

Index(['Neighborhood', 'year_month', 'burglary', 'danger of well-being',
       'domestic violence', 'fraud', 'homicide', 'property damage',
       'robbery with violence', 'robbery without violence', 'sexual crime',
       'threats'],
      dtype='object')

In [23]:
pre_data = df.set_index(["year_month","Neighborhood"]).unstack("Neighborhood")

### Train-test split


In [31]:
# Input lengh for the train-test split function and the ratio of the Folds

INPUT_LENGTH = 1 * 12 # records every 1 month x 12 months per year = 12 months
TRAIN_TEST_RATIO = 0.70 #70% of the data is going to be for training

#### Train-test-split Function

In [32]:
from typing import Tuple

def train_test_split(fold:pd.DataFrame,
                     train_test_ratio: float,
                     input_length: int) -> Tuple[pd.DataFrame]:
    """From a fold dataframe, take a train dataframe and test dataframe based on 
    the split ratio.
    - df_train should contain all the timesteps until round(train_test_ratio * len(fold))
    - df_test should contain all the timesteps needed to create all (X_test, y_test) tuples

    Args:
        fold (pd.DataFrame): A fold of timesteps
        train_test_ratio (float): The ratio between train and test 0-1
        input_length (int): How long each X_i will be

    Returns:
        Tuple[pd.DataFrame]: A tuple of two dataframes (fold_train, fold_test)
    """
    fold_train = fold[0:round(len(fold)*train_test_ratio)]
    fold_test = fold[(round(len(fold)*train_test_ratio - input_length)):]
    return fold_train,fold_test

In [33]:
# Running the T-T-Split function to get Fold_train and Fold_test

(fold_train, fold_test) = train_test_split(pre_data, TRAIN_TEST_RATIO, INPUT_LENGTH)

### (X_train, y_train) and (X_test, y_test)

#### get_X_y_strides Function


In [40]:
def get_X_y_strides(fold: pd.DataFrame, input_length: int, output_length: int, 
    sequence_stride: int) -> Tuple[np.array]:
    """slides through a `fold` Time Series (2D array) to create sequences of equal
        * `input_length` for X,
        * `output_length` for y,
    using a temporal gap `sequence_stride` between each sequence

    Args:
        fold (pd.DataFrame): One single fold dataframe
        input_length (int): Length of each X_i
        output_length (int): Length of each y_i
        sequence_stride (int): How many timesteps to take before taking the next X_i

    Returns:
        Tuple[np.array]: A tuple of numpy arrays (X, y)
    """
    #X, y = [], []

    for i in range(0, len(fold), sequence_stride):
        # Exits the loop as soon as the last fold index would exceed the last index
        if (i + input_length + output_length) >= len(fold):
            break
        X_i_transformed = fold.iloc[i:i + input_length, :]
        y_i_transformed = fold.iloc[i + input_length:i + input_length + output_length, :][TARGET]


        fold_train_list = X_i_transformed.stack("Neighborhood").groupby(["Neighborhood", "year_month"])\
                            .apply(lambda x: x.values.tolist()[0])\
                            .groupby("Neighborhood").apply(lambda x: x.values.tolist())\
                            .tolist()

        fold_test_list = y_i_transformed.stack("Neighborhood").groupby(["Neighborhood", "year_month"])\
                            .apply(lambda x: x.values.tolist()[0])\
                            .groupby("Neighborhood").apply(lambda x: x.values.tolist())\
                            .tolist()

    return (np.array(fold_train_list), np.array(fold_test_list))

In [41]:
# Defining the sequence stride, output length and target

SEQUENCE_STRIDE = 1
OUTPUT_LENGTH = 12
TARGET = ['burglary', 'danger of well-being',
       'domestic violence', 'fraud', 'homicide', 'property damage',
       'robbery with violence', 'robbery without violence', 'sexual crime',
       'threats']

In [42]:
# Running the Train function for X and y
X_train, y_train = get_X_y_strides(fold_train, INPUT_LENGTH, OUTPUT_LENGTH, SEQUENCE_STRIDE)

# Running the Test functeion for X and y
X_test, y_test = get_X_y_strides(fold_test, INPUT_LENGTH, OUTPUT_LENGTH, SEQUENCE_STRIDE)

print(X_train.shape, y_train.shape,X_test.shape, y_test.shape)

(2041, 12, 10) (2041, 12, 10) (2041, 12, 10) (2041, 12, 10)


## Model

In [43]:
def init_model(X_train, y_train):
    model = Sequential()

    # –– Model
    model.add(layers.Masking(mask_value=-1, input_shape=(12,10)))
    model.add(layers.LSTM(units=40, activation='tanh', return_sequences =True))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dropout(rate=0.2))  # The rate is the percentage of neurons that are "killed"
    model.add(layers.Dense(10, activation='relu'))

    # –– Compilation
    model.compile(loss='mse', 
                  optimizer='adam',
                 metrics = ["mae"]) 

    return model 

In [44]:
# Early Stopping with patience 10 
es = EarlyStopping(patience=10)

model = init_model(X_train, y_train)
model.summary()
history = model.fit(X_train, y_train, 
          epochs=200, 
          batch_size=32, 
          verbose=1, 
          callbacks = [es],
          validation_split=0.2)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_1 (Masking)         (None, 12, 10)            0         
                                                                 
 lstm_1 (LSTM)               (None, 12, 40)            8160      
                                                                 
 dense_2 (Dense)             (None, 12, 50)            2050      
                                                                 
 dropout_1 (Dropout)         (None, 12, 50)            0         
                                                                 
 dense_3 (Dense)             (None, 12, 10)            510       
                                                                 
Total params: 10720 (41.88 KB)
Trainable params: 10720 (41.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/200
51/51 

In [45]:
# Model Evaluation with test sets
model.evaluate(X_test,y_test)

64/64 [==============================] - 0s 919us/step - loss: 1.7671 - mae: 0.6851


[1.7670518159866333, 0.6851373314857483]

##  Predictions

In [46]:
predictions = model.predict(X_test)

64/64 [==============================] - 0s 710us/step


In [47]:
# Date Series for the prediction dataframe
start_date = '2023-01-01'
num_periods = 12

date_range = pd.date_range(start=start_date, periods=num_periods, freq='MS')
date_series = pd.Series(range(num_periods), index=date_range)

In [58]:
# Empty list to save the list of dataframes
p1 = []
nom_delitos = df.columns[2:]
nom_colonias = df.Neighborhood.unique()

for year_month in range(predictions.shape[1]):
    p1.append(pd.DataFrame(predictions[:, year_month, :], columns =nom_delitos, index = nom_colonias).assign(year_month= date_series.index[year_month].date()))

new_prediction = pd.concat(p1)

In [60]:
prediction_dataframe = new_prediction.set_index("year_month",append=True).round(0).astype(int)

In [61]:
prediction_dataframe

,,burglary,danger of well-being,domestic violence,fraud,homicide,property damage,robbery with violence,robbery without violence,sexual crime,threats
,year_month,,,,,,,,,,
ALVARO OBREGON || 16 DE SEPTIEMBRE,2023-01-01,0,0,0,0,0,0,0,0,0,0
ALVARO OBREGON || 19 DE MAYO,2023-01-01,0,0,0,0,0,0,0,0,0,0
ALVARO OBREGON || 1RA VICTORIA,2023-01-01,0,0,1,2,0,0,1,1,0,1
ALVARO OBREGON || 1RA VICTORIA SECCION BOSQUES,2023-01-01,0,1,2,1,0,0,1,1,0,1
ALVARO OBREGON || 26 DE JULIO,2023-01-01,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
XOCHIMILCO || VILLA XOCHIMILCO (U HAB),2023-12-01,0,0,0,0,0,0,0,0,0,0
XOCHIMILCO || VILLAS DE LOS TRABAJADORES (GDF) (U HAB),2023-12-01,0,0,0,0,0,0,0,0,0,0
XOCHIMILCO || VIVEROS DE COACTETLAN,2023-12-01,0,0,0,0,0,0,0,0,0,0


In [62]:
fold_test.reset_index()


year_month                           burglary  \
Neighborhood             ALVARO OBREGON || 16 DE SEPTIEMBRE   
0             2021-01-01                                  0   
1             2021-02-01                                  0   
2             2021-03-01                                  0   
3             2021-04-01                                  0   
4             2021-05-01                                  0   
5             2021-06-01                                  0   
6             2021-07-01                                  0   
7             2021-08-01                                  0   
8             2021-09-01                                  0   
9             2021-10-01                                  0   
10            2021-11-01                                  0   
11            2021-12-01                                  0   
12            2022-01-01                                  0   
13            2022-02-01                                  0   
14            2022-03-01                                  0   
15            2022-04-01                                  0   
16            2022-05-01                                  0   
17            2022-06-01                                  0   
18            2022-07-01                                  0   
19            2022-08-01                                  0   
20            2022-09-01                                  0   
21            2022-10-01                                  0   
22            2022-11-01                                  0   
23            2022-12-01                                  0   
24            2023-01-01                                  0   
25            2023-02-01                                  0   
26            2023-03-01                                  0   

                                                                          \
Neighborhood ALVARO OBREGON || 19 DE MAYO ALVARO OBREGON || 1RA VICTORIA   
0                                       1                              0   
1                                       0                              0   
2                                       0                              0   
3                                       0                              0   
4                                       0                              0   
5                                       0                              0   
6                                       0                              0   
7                                       0                              1   
8                                       0                              0   
9                                       0                              0   
10                                      1                              0   
11                                      0                              0   
12                                      1                              0   
13                                      0                              0   
14                                      0                              0   
15                                      0                              0   
16                                      0                              0   
17                                      0                              0   
18                                      0                              0   
19                                      0                              0   
20                                      0                              0   
21                                      0                              0   
22                                      0                              0   
23                                      0                              0   
24                                      0                              0   
25                                      0                              0   
26                                      0                 